# Naive Bayes (the easy way)

![image](images/001_spam.jpg)

![image](images/002_spam.jpg)

We'll cheat by using sklearn.naive_bayes to train a spam classifier! Most of the code is just loading our training data into a pandas DataFrame that we can play with:

In [2]:
import os
import io
import numpy
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [3]:
def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)


data = DataFrame({'message': [], 'class': []})

data = data.append(dataFrameFromDirectory('data/emails/spam', 'spam'))
data = data.append(dataFrameFromDirectory('data/emails/ham', 'ham'))

Let's have a look at that DataFrame:

In [4]:
data.head()

,class,message
data/emails/spam\00001.7848dde101aa985090474a91ec93fcf0,spam,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr..."
data/emails/spam\00002.d94f1b97e48ed3b553b3508d116e6a09,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
data/emails/spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,spam,1) Fight The Risk of Cancer!\n\nhttp://www.adc...
data/emails/spam\00004.eac8de8d759b7e74154f142194282724,spam,##############################################...
data/emails/spam\00005.57696a39d7d84318ce497886896bf90d,spam,I thought you might like these:\n\n1) Slim Dow...


CountVectorizer를 사용하여 각 메일의 메시지를 단어 리스트 (token)으로 분할. 이것을 MultinomialNB classifier를 학습. 

In [5]:
vectorizer = CountVectorizer()
counts = vectorizer.fit_transform(data['message'].values)
print(counts)

  (0, 20104)	1
  (0, 15629)	1
  (0, 30882)	1
  (0, 50553)	1
  (0, 36099)	1
  (0, 44217)	1
  (0, 58467)	1
  (0, 51216)	1
  (0, 10966)	1
  (0, 47038)	1
  (0, 46816)	1
  (0, 54656)	1
  (0, 43219)	2
  (0, 16635)	1
  (0, 38953)	1
  (0, 14434)	1
  (0, 16777)	1
  (0, 36134)	1
  (0, 35030)	1
  (0, 46819)	1
  (0, 12870)	1
  (0, 58727)	1
  (0, 22787)	1
  (0, 22197)	2
  (0, 53337)	2
  :	:
  (2999, 12870)	2
  (2999, 53337)	1
  (2999, 24806)	2
  (2999, 28034)	1
  (2999, 10697)	1
  (2999, 40368)	2
  (2999, 40892)	4
  (2999, 15189)	1
  (2999, 39016)	1
  (2999, 30375)	5
  (2999, 58736)	1
  (2999, 26143)	2
  (2999, 55923)	1
  (2999, 47451)	1
  (2999, 53220)	12
  (2999, 10746)	5
  (2999, 56678)	1
  (2999, 31369)	6
  (2999, 27728)	5
  (2999, 60793)	6
  (2999, 37734)	1
  (2999, 40661)	4
  (2999, 53745)	6
  (2999, 14755)	1
  (2999, 28855)	1


In [6]:
classifier = MultinomialNB()      # NaiveBayes Classifier
targets = data['class'].values
classifier.fit(counts, targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [7]:
counts

<3000x62964 sparse matrix of type '<class 'numpy.int64'>'
	with 429785 stored elements in Compressed Sparse Row format>

In [8]:
targets

array(['spam', 'spam', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

Let's try it out:

In [9]:
examples = ['Free coupon now!!!', "Hi Bob, how about a game of golf tomorrow?"]
example_counts = vectorizer.transform(examples)
predictions = classifier.predict(example_counts)
predictions

array(['spam', 'ham'], 
      dtype='<U4')

In [10]:
example_counts

<2x62964 sparse matrix of type '<class 'numpy.int64'>'
	with 11 stored elements in Compressed Sparse Row format>

## Activity

train/test to this spam classifier

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

In [12]:
cvectors = vectorizer.fit_transform(data['message'].values)
cvectors

<3000x62964 sparse matrix of type '<class 'numpy.int64'>'
	with 429785 stored elements in Compressed Sparse Row format>

In [13]:
y = data['class'].values
y

array(['spam', 'spam', 'spam', ..., 'ham', 'ham', 'ham'], dtype=object)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(cvectors, y, test_size=0.2)
print(X_train.shape)
print(X_test.shape)

(2400, 62964)
(600, 62964)


In [15]:
X_train

<2400x62964 sparse matrix of type '<class 'numpy.int64'>'
	with 350180 stored elements in Compressed Sparse Row format>

In [16]:
y_train

array(['spam', 'spam', 'ham', ..., 'spam', 'ham', 'ham'], dtype=object)

In [17]:
classifier = MultinomialNB()      # NaiveBayes Classifier
classifier.fit(X_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
y_pred = classifier.predict(X_test)
accuracy_score(y_test, y_pred)

0.96499999999999997

In [19]:
scores = cross_val_score(classifier, X_test, y_test, cv=5)
scores

array([ 0.92561983,  0.98347107,  0.95      ,  0.92436975,  0.90756303])